In [1]:
import numpy as np

Description
===========

This notebook is equivalent to the one used for the general survey [found here](../uband_depth/Lstar_depth_dust.ipynb) but applied to the DDFs specifically.

In [2]:
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles
from lsst.sims.maf.metrics import BaseMetric

import os
import sys
sys.path.append("../../LSST_OpSim/Scripts_NBs/")
from opsimUtils import *

import re

from DDF_utils import my_ddfInfo

#### Metrics

The functions here will set up the metrics. This function will be called immediately before running the runall on the bundleDict. It has built-in the search for existing files so calculations are only ran once. 

In [3]:
sys.path.append("../uband_depth/")
from ExgalM5_with_cuts_AGN import ExgalM5_with_cuts_AGN

In [4]:
#Use the same slicer for all. 
NSIDE = 64
my_slicer = slicers.HealpixSlicer(nside=NSIDE, useCache=False)

Healpix slicer using NSIDE=64, approximate resolution 54.967783 arcminutes


In [5]:
#Function to construct the metric bundles for a given OpSim, DDF and filter.
def get_mb(filters, DDF_names, opsdb, slicer, run, metricDataPath):     
    
    EM5 = list()
    bundleDict = dict()
    
    for filter in filters:
        for DDF_name in DDF_names:
            
            metricName = 'ExgalM5_with_cuts_AGN_{0}_{1}'.format(DDF_name, filter)
            metric = ExgalM5_with_cuts_AGN(lsstFilter=filter,
                                   metricName=metricName)
           
            constraint = 'filter = "{}"'.format(filter)
            constraintName = '{}'.format(filter)
        
            if DDF_name == 'AllDDFs':
                constraint += ' and proposalId > 1'
                constraintName += "_and_proposalId_gt_1"
            else:
                propids = my_ddfInfo(opsdb, DDF_name)['proposalId']
                if propids is None:
                    continue
                constraint += ' and ('
                constraintName += "_and_"
                for i, propid in enumerate(propids):
                    if i>0:
                        constraint += ' or '
                        constraintName += "_or_"
                    constraint += 'proposalId = {}'.format(propid)
                    constraintName += "proposalId_{}".format(propid)
                constraint += ')'
                
            #Check if this metric has already been run.
            runName = re.sub("\.","_",run)
            fname = "{0}/{1}_{2}_{3}_HEAL.npz".format(metricDataPath, 
                                                      runName, metricName, 
                                                      constraintName)
            if os.path.exists(fname):
                continue
            
            EM5.append(metricBundles.MetricBundle(metric, slicer, constraint))
            bundleDict['EM5_{0}_{1}'.format(DDF_name, filter)] = EM5[-1]
    
    return EM5, bundleDict

#### Filters and DDF fields

Set here the filters to use and the DDF fields on which to run the metric.

In [6]:
DDF_names = ['AllDDFs', 'ECDFS', 'COSMOS', 'XMM-LSS', 'ELAISS1', 'EDFS']
filters = ['u','g']

#### Set up folders

In [7]:
your_username = "rjassef"
folder_mafoutput = "DDFs_EM5_depths_{0:d}".format(NSIDE)

metricDataPath = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}/MetricData/'.format(
    your_username, folder_mafoutput)

resultDbPath  = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(
    your_username, folder_mafoutput)

outDir = '/home/idies/workspace/Storage/{0}/persistent/MAFOutput/{1}'.format(your_username,folder_mafoutput)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

#### Run function

In [10]:
#Function that runs all the metrics for all the OpSim, keeping track of the specific DDFs. 
def run_calcs(FBS_version, slicer, metricDataPath, filters, DDF_names):
    
    #OpSims folder. 
    dbDir = '/home/idies/workspace/lsst_cadence/FBS_{}/'.format(FBS_version)
    opSimDbs, resultDbs = connect_dbs(dbDir, outDir)
   
    #For each run, get all calculations.
    dbRuns = show_opsims(dbDir)
    for run in dbRuns:     
        
        print("Processing run ",run)
        
        #If OpSim run has no DDFs, then skip it. 
        opsim_propInfo = opSimDbs[run].fetchPropInfo()[1]
        #if len(opSimDbs[run].fetchPropInfo()[1]['DD']) == 0:
        if 'DD' not in opsim_propInfo.keys() or len(opsim_propInfo['DD'])==0:
            print("No DDFs in this run. Skipping to the next one.")
            continue

        #Set up the metric bundles for this run.
        EM5, bundleDict = get_mb(filters, DDF_names, opSimDbs[run], 
                                 slicer, run, metricDataPath)
        if len(EM5)==0:
            continue
            
        for EM5x in EM5:
            EM5x.setRunName(run)
        metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                        opSimDbs[run], metricDataPath, resultDbs[run])
        metricGroup.runAll()

#### Run for FBS 1.5

In [11]:
run_calcs("1.5", my_slicer, metricDataPath, filters, DDF_names)

Processing run  dcr_nham1_ugri_v1.5_10yrs
Processing run  rolling_mod6_sdf_0.20_v1.5_10yrs
Processing run  wfd_depth_scale0.95_noddf_v1.5_10yrs
No DDFs in this run. Skipping to the next one.
Processing run  u60_v1.5_10yrs
Processing run  footprint_stuck_rollingv1.5_10yrs
Processing run  filterdist_indx4_v1.5_10yrs
No DDFs in this run. Skipping to the next one.
Processing run  wfd_depth_scale0.70_noddf_v1.5_10yrs
No DDFs in this run. Skipping to the next one.
Processing run  daily_ddf_v1.5_10yrs
Processing run  alt_roll_mod2_dust_sdf_0.20_v1.5_10yrs
Processing run  goodseeing_gz_v1.5_10yrs
Processing run  filterdist_indx1_v1.5_10yrs
No DDFs in this run. Skipping to the next one.
Processing run  footprint_standard_goalsv1.5_10yrs
Processing run  footprint_big_sky_dustv1.5_10yrs
Processing run  footprint_bluer_footprintv1.5_10yrs
Processing run  twilight_neo_mod4_v1.5_10yrs
Processing run  goodseeing_gi_v1.5_10yrs
Processing run  footprint_big_wfdv1.5_10yrs
No proposal table available - n

/home/idies/lsst/stack/miniconda3-py37_4.8.2-cb4e2dc/Linux64/sims_maf/2.13.0.sims-64-g788d3062+53334d830b/python/lsst/sims/maf/maps/dustMap.py:25: UserWarning: Slicer value of nside (64) different from map value (128), using slicer value
  % (slicePoints['nside'],self.nside ))


Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and proposalId > 1 for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 12119 visits
Running:  ['EM5_AllDDFs_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "u" and (proposalId = 4) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 1176 visits
Running:  ['EM5_ELAISS1_u']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "u" and proposalId > 1 for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 6088 visits
Running:  ['EM5_AllDDFs_u']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint fil

/home/idies/lsst/stack/miniconda3-py37_4.8.2-cb4e2dc/Linux64/sims_maf/2.13.0.sims-64-g788d3062+53334d830b/python/lsst/sims/maf/metricBundles/metricBundleGroup.py:301: UserWarning: No data matching constraint filter = "u" and (proposalId = 4)
  warnings.warn('No data matching constraint %s' % constraint)
/home/idies/lsst/stack/miniconda3-py37_4.8.2-cb4e2dc/Linux64/sims_maf/2.13.0.sims-64-g788d3062+53334d830b/python/lsst/sims/maf/metricBundles/metricBundleGroup.py:305: UserWarning:  This means skipping metrics ['ExgalM5_with_cuts_AGN_EDFS_u :  u and (proposalId 4) : HealpixSlicer']
  warnings.warn(' This means skipping metrics %s' % metricsSkipped)


Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 5) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 781 visits
Running:  ['EM5_XMM-LSS_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 7) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 783 visits
Running:  ['EM5_ELAISS1_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "u" and (proposalId = 6) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 4052 visits
Running:  ['EM5_ECDFS_u']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 6) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth'

/home/idies/lsst/stack/miniconda3-py37_4.8.2-cb4e2dc/Linux64/sims_maf/2.13.0.sims-64-g788d3062+53334d830b/python/lsst/sims/maf/metricBundles/metricBundleGroup.py:301: UserWarning: No data matching constraint filter = "u" and (proposalId = 3)
  warnings.warn('No data matching constraint %s' % constraint)
/home/idies/lsst/stack/miniconda3-py37_4.8.2-cb4e2dc/Linux64/sims_maf/2.13.0.sims-64-g788d3062+53334d830b/python/lsst/sims/maf/metricBundles/metricBundleGroup.py:305: UserWarning:  This means skipping metrics ['ExgalM5_with_cuts_AGN_COSMOS_u :  u and (proposalId 3) : HealpixSlicer']
  warnings.warn(' This means skipping metrics %s' % metricsSkipped)


Found 848 visits
Running:  ['EM5_ELAISS1_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 3) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 1398 visits
Running:  ['EM5_COSMOS_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 4) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 880 visits
Running:  ['EM5_XMM-LSS_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 5 or proposalId = 6) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 1612 visits
Running:  ['EM5_EDFS_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Com

#### Run for FBS 1.6

In [ ]:
run_calcs("1.6", my_slicer, metricDataPath, filters, DDF_names)

Processing run  baseline_nexp2_scaleddown_v1.6_10yrs
Querying database SummaryAllProps with constraint filter = "u" and (proposalId = 5) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 1103 visits
Running:  ['EM5_XMM-LSS_u']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and proposalId > 1 for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 11194 visits
Running:  ['EM5_AllDDFs_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "u" and proposalId > 1 for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 5599 visits
Running:  ['EM5_AllDDFs_u']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 5) for colu

#### Run for FBS 1.7

In [ ]:
run_calcs("1.7", my_slicer, metricDataPath, filters, DDF_names)

Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "g" and (proposalId = 4) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 2220 visits
Running:  ['EM5_ELAISS1_g']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "u" and (proposalId = 2) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 1056 visits
Running:  ['EM5_XMM-LSS_u']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database SummaryAllProps with constraint filter = "u" and (proposalId = 4) for columns ['fieldDec', 'filter', 'fieldRA', 'fiveSigmaDepth']
Found 1064 visits
Running:  ['EM5_ELAISS1_u']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Processing run  rolling_scale0.6_nslice2_v1.7_10yr